<a href="https://colab.research.google.com/github/AmbiTyga/Task-Stylumia/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Kaggle Download
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c stumbleupon

Saving kaggle.json to kaggle.json
 86% 135M/157M [00:02<00:00, 54.8MB/s]
100% 157M/157M [00:02<00:00, 77.6MB/s]
  0% 0.00/21.5k [00:00<?, ?B/s]
100% 21.5k/21.5k [00:00<00:00, 20.6MB/s]
 62% 5.00M/8.02M [00:00<00:00, 23.2MB/s]
100% 8.02M/8.02M [00:00<00:00, 31.7MB/s]
  0% 0.00/3.45M [00:00<?, ?B/s]
100% 3.45M/3.45M [00:00<00:00, 220MB/s]


In [2]:
!unzip -q /content/raw_content.zip

In [3]:
!pip install boilerpy3 -q
!pip install scrapy -q

     |████████████████████████████████| 245kB 13.6MB/s 
     |████████████████████████████████| 245kB 20.4MB/s 
     |████████████████████████████████| 3.1MB 29.1MB/s 
     |████████████████████████████████| 3.2MB 66.6MB/s 
     |████████████████████████████████| 3.2MB 43.8MB/s 
     |████████████████████████████████| 61kB 5.6MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 81kB 6.8MB/s 


In [5]:
import pandas as pd 
import numpy as np
import json
from bs4 import BeautifulSoup
from boilerpy3 import extractors
import re

In [51]:
def extract_domain(df_url):
    # extract domains
    domain = df_url.str.lower().apply(lambda x:x.split('/')[2])
    domain_parts = domain.str.split('.')

    return domain_parts.apply(lambda x:'.'.join(x[-2:]) if x[-2] not in ['com', 'co'] else '.'.join(x[-3:]))

In [53]:
def clean_string(s):
    s = re.sub(r'[\r\t]', ' ',s)
    s = re.sub(r"\s+", ' ', s)
    return s.strip()

In [54]:
def boil_soup(urlid):
    filename = '/content/raw_content/' + urlid

    with open(filename, 'rb') as f:
        html = f.read()

        for parser in ["lxml", "xml", "html5lib"]:
            soup = BeautifulSoup(html, parser)
            if soup.body:
                return soup

        return BeautifulSoup(html)

In [55]:
data = pd.concat([pd.read_csv('/content/train.tsv.zip',sep = '\t'),
                  pd.read_csv('/content/test.tsv.zip',sep = '\t')],ignore_index = True)

In [56]:
item = data.copy()

In [57]:
item['real_url'] = item['url'].str.lower()

In [58]:
item['domain'] = extract_domain(item['real_url'])

In [59]:
item['tld'] = item['domain'].str.split('.').apply(lambda x:x[-1])

In [60]:
item['boilerplate'] = item['boilerplate'].apply(lambda x:json.loads(x))
item.rename(columns = {'url':'org_url'},inplace = True)

item = pd.concat([item,pd.DataFrame(item['boilerplate'].tolist(),columns=['title','body','url'])],axis = 1)
item.drop(columns = ['boilerplate'],inplace = True)

In [61]:
item['boilerplate'] = item[['title','body']].values.tolist()

In [ ]:
boiler = item['urlid'].astype(str).apply(get_boilerPipe)

## Snippets

In [85]:
soup = boil_soup('7018')

In [111]:
def get_boilerPipe(id):
  TAGS = ['title', 'h1', 'h2', 'h3', 'meta-description', 'meta-keywords',
          'img', 'a', 'other']
  TAGS_REMOVE = ['script', 'style']

  soup = boil_soup(id)

  ext = extractors.ArticleSentencesExtractor()

  for el in soup.find_all(TAGS_REMOVE):
    el.extract()
  VAL = {x:[] for x in TAGS}
  VAL['boilerpipe'] = ext.get_content_from_file('/content/raw_content/'+id)
  del ext
  for tag in TAGS:
    resultSet = soup.find_all(tag)
    if tag == 'img':
      try:
        VAL[tag] += [x['alt'] for x in resultSet]
      except KeyError:
        pass
      try:
        VAL[tag] += [x['title'] for x in resultSet]
      except KeyError:
        pass
    else:
      VAL[tag] += [x.text for x in resultSet]

  # extract meta tags
  meta = soup.find_all('meta')
  for el in meta:
    prop = el.get('property') if el.get('property') else el.get('name')
    if not prop:
      continue
    prop = prop.lower()
    try:
      s = str(el['content'])
    except:
      continue

    VAL['meta-'+prop] = s.split(u',') if prop == 'keywords' else [s]

  return VAL
